Parallel Schedule Generation Scheme (SGS)

The solver iteratively advances a simulation clock (current_time) and schedules jobs that become eligible. At each time step (or event), it considers:

* Job Completions: Processes jobs that finish at or before current_time, releasing their resources and marking them as completed.
* Eligibility Check: Identifies jobs whose predecessors are all finished and are not already scheduled or active.
* Resource Availability Check: For eligible jobs, it verifies if the required resources are available.
* Job Selection & Scheduling: If resources are available, it selects an eligible job based on a priority heuristic and schedules it to start at current_time.
* Clock Advancement: The current_time is advanced to the next significant event (either the next job completion or a unit time increment if no immediate completion occurs).

The algorithm maintains a dynamic available_resources dictionary and a list of active_jobs to track resource usage and job progress over time.

Heuristics Employed:

* Job Prioritization (LST + Penalty Weight): When multiple jobs are eligible and resources are available, the solver prioritizes jobs using a composite heuristic: (earliness_penalty + tardiness_penalty) / (max(0.1, slack) + 1.0).
* Latest Start Time (LST) Calculation: A backward pass is performed to calculate the Latest Start Time for each job, considering the common_due_date and durations. This is used to determine "slack" (how much a job can be delayed without impacting the due date).
* Composite Score: Jobs with higher combined penalties and lower slack (i.e., tighter deadlines or higher importance) receive a higher priority score, aiming to schedule them earlier. The max(0.1, slack) prevents division by zero and biases towards jobs with less slack.
* Earliest Feasible Start Time (Resource-Aware): The solver attempts to start a selected job at the current simulation time (current_time) if sufficient resources are available. If not, it waits until resources become free. This is a greedy approach to starting jobs as early as possible once they are eligible and resourced.

In [3]:
import json
import time
from typing import Dict, List, Any

def solve_project(json_data: Dict[str, Any]) -> Dict:
    """
    Robust solver for JIT-RCPSP with E/T penalties.
    Handles potential missing keys and varied resource ID types.
    """
    start_wall_time = time.perf_counter()
    
    # --- 1. Data Extraction & Normalization ---
    params = json_data.get('parameters', {})
    common_due_date = params.get('common_due_date', 0)
    jobs_list = json_data.get('jobs', [])
    
    # Safely extract core job data to avoid KeyErrors
    job_lookup = {}
    durations = {}
    predecessors = {}
    penalties = {}
    
    for j in jobs_list:
        j_id = j.get('id')
        if j_id is None: continue
        
        job_lookup[j_id] = j
        # Default to 0 if duration is missing
        durations[j_id] = j.get('duration', 0)
        predecessors[j_id] = j.get('predecessors', [])
        
        # Ensure penalties sub-dict exists
        p = j.get('penalties', {})
        penalties[j_id] = {
            'earliness': p.get('earliness_unit_penalty', 0),
            'tardiness': p.get('tardiness_unit_penalty', 0)
        }

    # Normalize Resource Registry (keys as strings)
    resource_registry = {}
    for r in json_data.get('resources', []):
        r_id = str(r.get('id', ''))
        resource_registry[r_id] = r.get('capacity', 0)

    # --- 2. Priority Calculation (LST + Penalty) ---
    # Backward pass for Latest Start Time (LST)
    successors = {j_id: [] for j_id in job_lookup}
    for j_id, preds in predecessors.items():
        for p_id in preds:
            if p_id in successors:
                successors[p_id].append(j_id)

    lst = {j_id: common_due_date - durations[j_id] for j_id in job_lookup}
    
    # Iterate to propagate precedence constraints
    # (Doing this multiple times ensures convergence for deep DAGs)
    for _ in range(len(job_lookup)):
        changed = False
        for j_id in job_lookup:
            if successors[j_id]:
                min_succ_start = min(lst[s_id] for s_id in successors[j_id])
                new_lst = min(lst[j_id], min_succ_start - durations[j_id])
                if new_lst != lst[j_id]:
                    lst[j_id] = new_lst
                    changed = True
        if not changed: break

    def get_priority_score(j_id):
        # Higher score = schedule sooner
        weight = penalties[j_id]['earliness'] + penalties[j_id]['tardiness']
        # Slack is how much we can delay before hitting the due date
        slack = lst[j_id]
        return weight / (max(0.1, slack) + 1.0)

    # --- 3. Parallel Schedule Generation Scheme ---
    schedule = {}
    finished_ids = set()
    active_jobs = [] # (finish_time, job_id)
    available_resources = resource_registry.copy()
    current_time = 0
    
    while len(schedule) < len(job_lookup):
        # Process completions
        active_jobs.sort()
        while active_jobs and active_jobs[0][0] <= current_time:
            _, j_id = active_jobs.pop(0)
            reqs = job_lookup[j_id].get('resources_required', {})
            for r_id, qty in reqs.items():
                available_resources[str(r_id)] += qty
            finished_ids.add(j_id)

        # Identify and Sort Eligible Jobs
        eligible = [
            j_id for j_id in job_lookup
            if j_id not in schedule 
            and j_id not in [x[1] for x in active_jobs]
            and all(p_id in finished_ids for p_id in predecessors[j_id])
        ]
        eligible.sort(key=get_priority_score, reverse=True)

        # Resource Allocation
        any_started = False
        for j_id in eligible:
            reqs = job_lookup[j_id].get('resources_required', {})
            if all(available_resources.get(str(r_id), 0) >= qty for r_id, qty in reqs.items()):
                # Start job
                for r_id, qty in reqs.items():
                    available_resources[str(r_id)] -= qty
                schedule[j_id] = current_time
                active_jobs.append((current_time + durations[j_id], j_id))
                any_started = True
        
        # Advance Clock
        if active_jobs:
            # Advance to next finish event
            next_time = min(f_time for f_time, _ in active_jobs)
            current_time = max(current_time + 1, next_time) if not any_started else current_time
        else:
            current_time += 1

    # --- 4. Final Metrics ---
    total_penalty = 0.0
    makespan = 0
    for j_id, start in schedule.items():
        finish = start + durations[j_id]
        makespan = max(makespan, finish)
        if finish < common_due_date:
            total_penalty += (common_due_date - finish) * penalties[j_id]['earliness']
        elif finish > common_due_date:
            total_penalty += (finish - common_due_date) * penalties[j_id]['tardiness']

    return {
        "schedule": schedule,
        "metrics": {
            "makespan": makespan,
            "total_penalty": round(total_penalty, 2),
            "execution_time": f"{(time.perf_counter() - start_wall_time)*1000:.4f}s"
        }
    }

In [5]:
with open("C:/Users/Admin/Desktop/sch50.json", 'r') as f:
    data = json.load(f)  
result = solve_project(data)
print(result)

{'schedule': {20: 0, 6: 0, 8: 0, 21: 0, 27: 0, 10: 0, 11: 0, 39: 0, 47: 0, 9: 0, 23: 0, 28: 0, 30: 0, 31: 0, 49: 0, 2: 0, 3: 0, 22: 0, 32: 0, 33: 0, 41: 0, 1: 0, 15: 0, 44: 0, 14: 0, 29: 0, 16: 0, 40: 0, 42: 0, 19: 0, 25: 0, 38: 0, 45: 0, 46: 0, 5: 0, 13: 0, 24: 0, 43: 0, 4: 0, 36: 0, 50: 0, 17: 0, 37: 0, 48: 0, 18: 0, 12: 0, 26: 0, 35: 0, 7: 0, 34: 0}, 'metrics': {'makespan': 0, 'total_penalty': 27392.0, 'execution_time': '1.9291s'}}
